In [2]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\daniel.martinezcarre\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [3]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import random


In [4]:
def waitFinishLoad(driver):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "page-loader")))
    WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.ID, "page-loader")))
    alea=random.choice([0.5, 0.1, 0.2, 0.3, 0.4, 0.6])
    time.sleep(alea)

def carga(driver):
    WebDriverWait(driver, 5, 1)

def clickByPartialText(auxText, driver):
    waitFinishLoad(driver)
    elem = driver.find_element_by_partial_link_text(auxText)
    elem.click()
    waitFinishLoad(driver)

def isThisLastPage(driver):
    if (getLastPage(driver) == getCurrentPage(driver)):
        return True
    return False



### Variables y funciones

In [5]:
URL="https://contrataciondelestado.es/"

In [36]:
def esperar_carga_pagina(driver, url):
    """
    Espera a que la página web se cargue completamente.

    Args:
        driver: El objeto del navegador web.
        url: La URL de la página web que se quiere cargar.
    """
    # Carga la página web
    driver.get(url)

    # Espera a que la página se cargue completamente
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")


def automatizaciones(driver):
    """
    Ejecuta las automatizaciones en la página web con esperas.

    Args:
        driver: El objeto del navegador web.
    """

    # Click en 'Publicaciones'
    wait = WebDriverWait(driver, 10)
    link = wait.until(EC.visibility_of_element_located((By.XPATH, "//a[@href='/wps/portal/licitaciones']")))
    link.click()

    # Click en 'Licitaciones'
    wait = WebDriverWait(driver, 10)
    link_licitaciones = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "divLogo")))
    link_licitaciones.click()

    # Click en 'Búsqueda avanzada'
    wait = WebDriverWait(driver, 10)
    link_busqueda_avanzada = wait.until(EC.presence_of_element_located((By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textBusquedaAvanzada")))
    link_busqueda_avanzada.click()

    # Click en 'Seleccionar'
    wait = WebDriverWait(driver, 10)
    link_busqueda_avanzada = wait.until(EC.presence_of_element_located((By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:idSeleccionarOCLink")))
    link_busqueda_avanzada.click()

    # Click en 'Entidades'
    wait = WebDriverWait(driver, 10)
    entidades = wait.until(EC.presence_of_element_located((By.XPATH, "//div[text()='ENTIDADES LOCALES']/../../td[@class='multiline']")))
    entidades.click()

    # Click en 'Galicia'
    wait = WebDriverWait(driver, 10)
    galicia = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[text()='Galicia']/../../td[@class='multiline']")))
    galicia[1].click()

    # Click en 'A Coruña'
    wait = WebDriverWait(driver, 10)
    link_coruna_1 = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[text()='A Coruña']/../../td[@class='multiline']")))
    link_coruna_1[2].click()

    # Click en 'Ayuntamientos'
    wait = WebDriverWait(driver, 10)
    link_junta = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[text()='Ayuntamientos']/../../td[@class='multiline']")))
    link_junta[3].click()

    # Click en 'A Coruña' por 2ª vez
    wait = WebDriverWait(driver, 10)
    link_junta = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[text()='A Coruña']")))
    link_junta[1].click()

    # Click en 'Junta de Gobierno del Ayuntamiento de A Coruña'
    wait = WebDriverWait(driver, 10)
    link_junta = wait.until(EC.presence_of_element_located((By.XPATH, "//option[text()='Junta de Gobierno del Ayuntamiento de A Coruña']")))
    link_junta.click()

    # Click en 'Añadir'
    wait = WebDriverWait(driver, 10)
    boton_anhadir = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@name='viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:botonAnadirMostrarPopUpArbolEO']")))
    boton_anhadir.click()

    # Click en 'Buscar'
    wait = WebDriverWait(driver, 10)
    boton_buscar = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@title='Buscar']")))
    boton_buscar.click()



### Carga de la URL y automatizaciones

In [37]:
driver = webdriver.Firefox()
esperar_carga_pagina(driver, URL)
automatizaciones(driver)

### Parseado de tabla e introducción de valores con BeautifulSoup4

In [45]:
import requests
from bs4 import BeautifulSoup
import pprint

In [47]:
# Saco la URL actual
url_actual = driver.current_url
page = requests.get(url_actual)

soup = BeautifulSoup(page.content, "html.parser")
pprint.pprint(soup.prettify())

('<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" '
 '"http://www.w3.org/TR/html4/loose.dtd">\n'
 '<!-- I231220_010 - PETICION DE SERVICIO: cambiar el logo en PRE1, PRE2 y '
 'PROD -->\n'
 '<html lang="es">\n'
 ' <script language="javascript" type="text/javascript">\n'
 '  /*\r\n'
 ' * I190531_010 - PLACE PRO: Estudio sobre posible causa de peticiones '
 'repetidas\r\n'
 ' * P190607_001 - PLACE PRO: Problema correctivo para solucionar peticiones '
 'repetidas.\r\n'
 ' *\r\n'
 ' * Aplicar este script a todas las paginas que contengan '
 '"<hx:scriptCollector>"\r\n'
 ' * Incluido en Default.jsp\r\n'
 ' *             ...\r\n'
 ' *             \r\n'
 ' *             \r\n'
 ' *             <body ...\r\n'
 '*/\r\n'
 '\r\n'
 '/* applyModKeyBoardEvents\r\n'
 ' * Indica a que paginas se les modifica el comportamiento por defecto \r\n'
 ' * de los eventos onkeydown, onkeypress y onkeyup en los elementos <form '
 '...\r\n'
 ' * "noApply" - No se aplica a ninguna pagina\r\n'
 ' * "s

In [50]:
tabla = soup.find("table", id="myTablaBusquedaCustom")
# filas_1 = soup.find("tr", class_="rowClass1")
# filas_2 = soup.find("tr", class_="rowClass2")

pprint.pprint(tabla)

None
